In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os, time, json, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import data_preprocessing as dp
import backtrader as bt
import matplotlib.pyplot as plt

## Get data

In [ ]:
frequency = timedelta(seconds=60)
pair = 'USDT_BTC'
date_start = '2020-11-11'
date_end = '2021-03-31'
lob_depth = 100
norm_type = 'dyn_z_score'
roll = 1440*10 # 10 days

In [ ]:
df_data, df_data_stdz = dp.import_data(
    pair, 
    date_start, 
    date_end, 
    frequency=frequency, 
    depth=lob_depth, 
    norm_type=norm_type, 
    roll=roll, 
    stdz_depth=1
)

In [ ]:
df_data['Mid_Price'].plot()

## Resample

In [ ]:
df_data[['Mid_Price']].head(31)

In [ ]:
# resample data to a less granular frequency
df_data = df_data.asfreq('1min')
df_data['volume'] = df_data['amount_buy'] + df_data['amount_sell']

data_resampled = df_data.resample('30min', label='right').agg( # closing time of candlestick
    {
    'Mid_Price': ['last', 'first', np.max, np.min], 
    'volume': np.sum
    }
)

data_resampled.columns = data_resampled.columns.get_level_values(1)

data_resampled['close'] = data_resampled['last']
data_resampled['open'] = data_resampled['first']
data_resampled['high'] = data_resampled['amax']
data_resampled['low'] = data_resampled['amin']
data_resampled['volume'] = data_resampled['sum']
data_resampled.index.name = 'datetime'

data_resampled
# rename columns

In [ ]:
data_resampled['log_ret'] = (np.log(data_resampled['close']) - np.log(data_resampled['close'].shift(1)))
data_resampled['roll_std'] = data_resampled['log_ret'].rolling(window=336).std() # 336 is the number of 30mins interval in week
data_resampled['roll_std'].plot(figsize=(8,4))

## Backtrader

In [ ]:
from Strategies.GoldenCross import GoldenCross
from Strategies.BuyHold import BuyHold

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(GoldenCross)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=data_resampled[:2000])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addwriter(bt.WriterFile, out='./Strategies/logging/golden_cross2.csv', csv=True)

# Set our desired cash start
cerebro.broker.setcash(200000.0)
# Add a FixedSize sizer according to the stake
# cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
# cerebro.broker.setcommission(commission=0.0007) 

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything


cerebro.run()

plt.rcParams['figure.figsize']=[22, 16]
cerebro.plot()
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# figure out what's wrong with stop losses


In [ ]:
strategy_results = pd.read_csv('./Strategies/logging/golden_cross2.csv', header=1, index_col='Id').dropna(thresh=3)
strategy_results['datetime'] = pd.to_datetime(strategy_results['datetime'])
print(strategy_results.shape)

In [ ]:
strategy_and_indic = pd.merge(data_resampled, strategy_results, left_index=True, right_on='datetime', how='outer')
print(strategy_and_indic.columns)
columns_to_keep = ['datetime', 'open_x', 'close_x', 'high_x', 'low_x', 'cash', 'value', 'buy', 'sell', 'pnlplus', 'pnlminus', 'sma', 'sma.1', 'crossover']
strategy_and_indic[columns_to_keep].to_csv('./Strategies/logging/golden_cross_cl.csv')

In [ ]:
# def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
#              width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

#         from backtrader import plot
#         if cerebro.p.oldsync:
#             plotter = plot.Plot_OldSync(**kwargs)
#         else:
#             plotter = plot.Plot(**kwargs)

#         figs = []
#         for stratlist in cerebro.runstrats:
#             for si, strat in enumerate(stratlist):
#                 rfig = plotter.plot(strat, figid=si * 100,
#                                     numfigs=numfigs, iplot=iplot,
#                                     start=start, end=end, use=use)
#                 figs.append(rfig)

#         for fig in figs:
#             for f in fig:
#                 f.savefig(file_path, bbox_inches='tight')
#         # return figs

# saveplots(cerebro, file_path = 'savefig.png') 

## My trading functions

In [ ]:
## Roadmap
# for each trade I need entry price, closing price, number of periods, time in the trade, min, max, volatility V 
# make execution assumptions: conservative: enter trade next open bar, exit trade next open bar V
# add stops and trailing stops - in progress
# wrap strategy in a reusable class - in progress
# pull more data, a few pairs and recent data
# add single strategy to binance account with cctx
# backtest multiple strategies across multiple pairs, splitting between train and test set etc
# deploy multiple strategies

In [ ]:
import ccxt
import ta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import EMAIndicator
import config
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
## Exchange connectivity
exchange = ccxt.binance(
    {
        'apiKey': config.BINANCE_API_KEY,
        'secret': config.BINANCE_SECRET_KEY
    }
)

markets = exchange.load_markets()

bars = exchange.fetch_ohlcv('ETH/USDT', limit=20) # most recent candle keeps evolving

In [ ]:
from StratTest.engine import TradingStrategy

In [ ]:
trading_strategy = TradingStrategy(data_resampled[['open', 'high', 'low', 'close', 'volume']].copy())

# trading_strategy.add_indicator('BollingerBands', window=20)
trading_strategy.add_indicator('EMAIndicator', window=20)
trading_strategy.add_indicator('EMAIndicator', window=50)

In [ ]:
trading_strategy.add_strategy(
    'EMACrossOver', 
    execution_type='next_bar_open',
    stop_loss=0.01,
    short_ema='ema_20', 
    long_ema='ema_50'
)

In [ ]:
# TODO delete _new_osition, _trades, _signal columns, now the same as the ones with strategy
# count number of transaction per period and add transaction costs
# proceede with more data

In [ ]:
 # check what is going on on March 22
trading_strategy.trading_chart(plot_strategy=True, short_ema='ema_20', long_ema='ema_50')

In [ ]:
trading_strategy.trading_chart(plot_strategy=True, short_ema='ema_20')

In [ ]:
trading_strategy.trading_chart(plot_strategy=True, short_ema='ema_20')

In [ ]:
trading_strategy.df.to_excel(f'StratTest/Exports/{trading_strategy.strategy}_{trading_strategy.stop_loss}.xlsx')

In [ ]:
# df =data_resampled[['open', 'high', 'low', 'close', 'volume']].copy() # pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# ## Create Indicators
# # Bollinger Bands
# bb_indicator = BollingerBands(df['close'], window=20)

# df['upper_band'] = bb_indicator.bollinger_hband()
# df['lower_band'] = bb_indicator.bollinger_lband()
# df['moving_average'] = bb_indicator.bollinger_mavg()

# # Average True Range
# atr_indicator = AverageTrueRange(df['high'], df['low'], df['close'])
# df['atr'] = atr_indicator.average_true_range()

# # Moving Averages
# ema50_indicator = EMAIndicator(df['close'], 50)
# df['ema_50'] = ema50_indicator.ema_indicator()

# ema20_indicator = EMAIndicator(df['close'], 20)
# df['ema_20'] = ema20_indicator.ema_indicator()

In [ ]:
# ## Generate Signals
# # EMA cross
# df['ema_cross_signal'] = np.where(
#     df['ema_20'] > df['ema_50'], 1, 
#     np.where(df['ema_20'] < df['ema_50'], -1, 0))

# df['ema_cross_trades'] = np.where(
#     df['ema_cross_signal'].diff() > 0, 'buy', 
#     np.where(df['ema_cross_signal'].diff() < 0, 'sell', 'hold'))

# df['ema_cross_new_position'] = np.where(
#     df['ema_cross_signal'].diff() > 0, +1, 
#     np.where(df['ema_cross_signal'].diff() < 0, -1, 0))

In [ ]:
# ## Backtesting
# initial_cash = 1000

# df['returns'] = np.log(df['close']) - np.log(df['close'].shift(1))
# df['ema_cross_returns'] = df['returns'] * df['ema_cross_signal']

# df['ema_cross_cum_performance'] = np.exp(df['ema_cross_returns'].cumsum())
# df['ema_cross_cash'] = df['ema_cross_cum_performance'] * initial_cash

# np.exp(df['returns'].cumsum()).plot(figsize=(8,4), legend=True) # reverse log returns to prices
# df['ema_cross_cum_performance'].plot(legend=True)

In [ ]:

# trading_strategy.df['px_returns_calcs']  = np.where(
#     trading_strategy.df['EMACrossOver_new_position']!=0, trading_strategy.df['open'].shift(-1), trading_strategy.df['close'])

# finish up creating the columns with actual execution prices to be used to calculate strategy returns

In [ ]:
trading_strategy.df['px_returns_calcs'].plot()

In [ ]:
# # potential entry points

# df['potential_entry_price'] = df['open'].shift(-1) # assume entry trade is executed at the next bar open
# df['potential_closing_price'] = df['open'].shift(-1) # assume closing is executed at the next bar open
# df[['open', 'potential_entry_price']]

# # get positions in the dataframe where indicator generates signals
# open_trades_idx = np.where(df['ema_cross_new_position']!=0)[0]
# # -2 because of shape is n rows and df is 0 indexed and because we do + 1 later - avoid out of bound error
# closing_trades_idx = np.append(open_trades_idx, df.shape[0]-2)[1:] 

# df['trade_grouper'] = np.nan
# df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'] = df.iloc[open_trades_idx].index
# df['trade_grouper'] = df['trade_grouper'].fillna(method='ffill')

# # scenario where no stop loss is present, invested position is the same as the signal output
# df['strategy_new_position'] = df['ema_cross_new_position'].copy()
# df['strategy_trades'] = df['ema_cross_trades'].copy()
# df['strategy_signal'] = df['ema_cross_signal'].copy()

# # col to keep track of stop loss trigger
# df['sl_trigger'] = np.nan
# df['sl_hit'] = np.nan

# all_trades_list = []
# for name, sub_df in df.groupby(by='trade_grouper'):

#     entry_price = df[df.index==name]['potential_entry_price'].values[0]
#     direction = df[df.index==name]['ema_cross_new_position'].values[0]
#     print(direction)

#     # check for stop losses before any backtesting
#     if direction > 0:

#         sl_price = entry_price * 0.99
#         sub_df['sl_trigger'] = sl_price
#         df.loc[sub_df.index, 'sl_trigger'] = sl_price

#         if (sub_df['sl_trigger'] < sub_df['low']).sum() == sub_df.shape[0]:
#             print('Long position held until signal reversed')
#         else:
#             sl_trigger_time = sub_df[~(sub_df['sl_trigger'] < sub_df['low'])].index[0] # when stop loss was triggered
#             sl_affected_range = sub_df[sub_df.index>=sl_trigger_time].index # all the datapoints subsequently affected by stop loss

#             df.loc[sl_trigger_time, 'strategy_new_position'] = -1 # create exit point when sl is hit
#             df.loc[sl_trigger_time, 'strategy_trades'] = "sell" # create exit point when sl is hit
#             df.loc[sl_trigger_time, 'sl_hit'] = True # flag stop loss being hit
#             df.loc[sl_affected_range, 'strategy_signal'] = 0 # turn signal to 0 - out of market
            
#             print('Stop loss triggered - closing long position')

#     elif direction < 0:

#         sl_price = entry_price * 1.01
#         sub_df['sl_trigger'] =  sl_price
#         df.loc[sub_df.index, 'sl_trigger'] = sl_price

#         if (sub_df['sl_trigger'] > sub_df['high']).sum() == sub_df.shape[0]:
#             print('Short position held until signal reversed')
#         else:
#             sl_trigger_time = sub_df[~(sub_df['sl_trigger'] > sub_df['high'])].index[0] # when stop loss was triggered
#             sl_affected_range = sub_df[sub_df.index>=sl_trigger_time].index  # all the datapoints subsequently affected by stop loss

#             df.loc[sl_trigger_time, 'strategy_new_position'] = 1 # create exit point when sl is hit
#             df.loc[sl_trigger_time, 'strategy_trades'] = "buy" # create exit point when sl is hit
#             df.loc[sl_trigger_time, 'sl_hit'] = True # flag stop loss being hit
#             df.loc[sl_affected_range, 'strategy_signal'] = 0 # turn signal to 0 - out of market
            
#             print('Stop loss triggered - closing short position')

In [ ]:
df[df.index>='2020-11-22 21:00:00'].head(50)[['trade_grouper', 'trade_grouper', 'close', 'low', 'high', 'sl_trigger', 'ema_cross_new_position', 'ema_cross_signal', 'ema_cross_trades', 'strategy_new_position', 'strategy_signal', 'strategy_trades', 'sl_hit']]

In [ ]:
# get positions in the dataframe where indicator generates signals
open_trades_idx = np.where(df['strategy_position']!=0)[0]
df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'].head(20)

In [ ]:
# prepare df trades

# get positions in the dataframe where indicator generates signals
open_trades_idx = np.where(df['ema_cross_position']!=0)[0]
# -2 because of shape is n rows and df is 0 indexed and because we do + 1 later - avoid out of bound error
closing_trades_idx = np.append(open_trades_idx, df.shape[0]-2)[1:] 
df_trades = df.iloc[open_trades_idx][['ema_cross_position']].copy() # empty dataframe with only datetime index

# entry and closing points
df_trades['entry_price'] = df.iloc[open_trades_idx+1]['open'].values # assume entry trade is executed at the next bar open
df_trades['closing_price'] = df.iloc[closing_trades_idx+1]['open'].values # assume closing is executed at the next bar open

# trade discrete returns
df_trades['discrete_return'] = df_trades['ema_cross_position'] * ((df_trades['closing_price'] / df_trades['entry_price']) - 1)

# how long are the trades 
df_trades['trade_n_periods'] = closing_trades_idx - open_trades_idx
df_trades['trade_duration'] = df.iloc[closing_trades_idx].index - df.iloc[open_trades_idx].index

# what happened throughout the trade
df['trade_grouper'] = np.nan
df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'] = df.iloc[open_trades_idx].index
df['trade_grouper'] = df['trade_grouper'].fillna(method='ffill')
df.head(60)

all_trades_list = []
for name, sub_df in df.groupby(by='trade_grouper'):
    max_val = sub_df['high'].max()
    min_val = sub_df['low'].min()
    returns_std = sub_df['returns'].std()

    all_trades_list.append([name, max_val, min_val, returns_std])


intra_trade_stats = pd.DataFrame(all_trades_list, columns=['datetime', 'px_high', 'px_low', 'returns_std']).set_index('datetime')
df_trades = pd.merge(df_trades, intra_trade_stats, left_index=True, right_index=True)


def max_dd_pctg(row):
    ''' Measure of how "painful" holding the trade was '''
    if row['ema_cross_position'] == 1:
        return (row['entry_price'] - row['px_low'])/row['px_low']
    elif row['ema_cross_position'] == -1:
        return (-(row['entry_price'] - row['px_high']))/row['px_high']
    else:
        return 0

df_trades['dd_pctg'] = df_trades.apply(max_dd_pctg, axis=1)

# calculate trade returns and jump into risk management / stop losses


In [ ]:

sl_trigger_time = sub_df[~(sub_df['sl_trigger'] < sub_df['low'])].index

# shortened trade time due to stop loss
stopped_sub_df = sub_df[sub_df.index<=sl_trigger_time[0]].copy()
stopped_sub_df['strategy_position'][-1] = -1

# remaining part of the trade, now position need to change to 0
quitted_sub_df = sub_df[sub_df.index>=sl_trigger_time[0]].copy()
quitted_sub_df

In [ ]:
sub_df.loc[sl_trigger_time, 'strategy_position'] = -1
sub_df


In [ ]:
sub_df[sub_df.index<=sl_trigger_time[0]]['strategy_position'][-1] = -1
sub_df[sub_df.index<=sl_trigger_time[0]]['strategy_position']

In [ ]:
def get_worst_price(row):
    ''' Get worst price relative to position '''
    if row['ema_cross_signal'] > 0:
        return min(row['open'], row['high'], row['low'], row['close'])
    elif row['ema_cross_signal'] < 0:
        return max(row['open'], row['high'], row['low'], row['close'])
    else:
        return 0



# static stop

sub_df['worst_price_timestamp'] = sub_df.apply(get_worst_price, axis=1)
# calculate loss vs worst price over the period
sub_df['cumulative_performance'] = sub_df['ema_cross_returns'].cumsum()
sub_df['worst_period_potential_loss'] = sub_df['ema_cross_signal'] * ((sub_df['worst_price_timestamp'] / entry_price) - 1)

sub_df[['ema_cross_returns', 'cumulative_performance', 'worst_period_potential_loss']]

In [ ]:
df_trades.apply(lambda x: (x['closing_price'] / x['entry_price']) - 1)

In [ ]:
## Metrics
# Net Profit
net_profit = df['ema_cross_cash'][-1] - initial_cash 

# Max Drowdown
max_dd = df_trades['dd_pctg'].max()

# Win Ratio
win_ratio = (df_trades['discrete_return']>0).sum() / df_trades.shape[0]

print(f'Net Profit: {net_profit:.2f}, Max Drawdown: {max_dd:.2%}, Win Ratio: {win_ratio:.2%}')

In [ ]:
# # df['close'].plot(legend=True)
# ((np.exp(df['ema_cross_returns'].cumsum()) * 100)).plot(legend=True)
# # ((np.exp(df['returns'].cumsum()) * df['close'][0])).plot(legend=True)

In [ ]:
# df['ema_cross_position'].cumsum().plot()
# df['ema_cross_signal'].plot()

In [ ]:
## Plotting
plot_indic_list = ['ema_50', 'ema_20']
plot_indic_color = ['#CCFFFF', '#FFCCFF']

fig = make_subplots(
    rows=2, 
    cols=1,
    shared_xaxes=True,
    row_heights=[0.2, 0.8],
    vertical_spacing=0.02
)

fig.add_trace(
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='px',
        increasing_line_color= 'green', 
        decreasing_line_color= 'red'
    ),
    row=2, 
    col=1
)
# candlestick xaxes
fig.update_xaxes(rangeslider_visible=False,    row=2,
    col=1)


# add indicators to candlestick chart
for indic, color in zip (plot_indic_list, plot_indic_color):
    fig.add_scatter(
        x=df.index, 
        y=df[indic], 
        name=indic, 
        marker=dict(color=color),
        row=2, 
        col=1
    )

# add buy trades marks
fig.add_scatter(
    x=df.index, 
    y=df['close']+100, 
    showlegend=False,
    # name='trades', 
    mode='markers',
    marker=dict(
        size=12,
        # I want the color to be green if 
        # lower_limit ≤ y ≤ upper_limit
        # else red
        color=(
            (df['ema_cross_trades'] == 'buy')).astype('int'),
        colorscale=[[0, 'rgba(255, 0, 0, 0)'], [1, '#B7FFA1']],
        symbol=5
    ),
    row=2, 
    col=1
)

# add sell trades marks
fig.add_scatter(
    x=df.index, 
    y=df['close']-100, 
    showlegend=False,
    # name='trades', 
    mode='markers',
    marker=dict(
        size=12,
        # I want the color to be green if 
        # lower_limit ≤ y ≤ upper_limit
        # else red
        color=(
            (df['ema_cross_trades'] == 'sell')).astype('int'),
        colorscale=[[0, 'rgba(255, 0, 0, 0)'], [1, '#FF7F7F']],
        symbol=6   
        ),
        row=2, 
        col=1
)

# add strategy returns
fig.add_scatter(
    x=df.index,
    y=df['ema_cross_cum_performance'],
    name='cum_performance',
    row=1,
    col=1
)

# general layout
fig.update_layout(
    width=1400,
    height=600,
    title='<b>Strategy</b>',
    title_x=.5,
    yaxis_title='USDT/BTC',
    template="plotly_dark",
    # plot_bgcolor='rgb(10,10,10)'
)

fig.show()

In [ ]:
df[df.index>='2020-11-22 21:00:00'].head(50)[['trade_grouper', 'trade_grouper', 'close', 'low', 'high', 'sl_trigger', 'ema_cross_new_position', 'ema_cross_signal', 'ema_cross_trades', 'strategy_new_position', 'strategy_signal', 'strategy_trades']]